<a href="https://colab.research.google.com/github/Junhojuno/DeepLearning-Beginning/blob/master/tf_estimator_linearmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.feature_column as fc

import os
import sys

import matplotlib.pyplot as plt
from IPython.display import clear_output

In [0]:
tf.enable_eager_execution()

In [3]:
!pip install -q requests
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3168, done.
remote: Counting objects: 100% (3168/3168), done.
remote: Compressing objects: 100% (2682/2682), done.
remote: Total 3168 (delta 573), reused 2075 (delta 408), pack-reused 0
Receiving objects: 100% (3168/3168), 370.56 MiB | 40.54 MiB/s, done.
Resolving deltas: 100% (573/573), done.
Checking out files: 100% (2998/2998), done.


In [0]:
# add the root directory of the repo to your python path
models_path = os.path.join(os.getcwd(), 'models')
sys.path.append(models_path)

In [5]:
# download the dataset
from official.wide_deep import census_dataset, census_main
census_dataset.download("/tmp/census_data/")

W0618 02:17:59.553497 140493422073728 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:78: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0618 02:17:59.555037 140493422073728 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:81: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0618 02:18:00.228131 140493422073728 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:62: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

W0618 02:18:00.459901 140493422073728 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:73: The name tf.gfile.Remove is deprecated. Please use tf.io.gfile.remove instead.



In [0]:
# for using in Command line
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + models_path
else:
    os.environ['PYTHONPATH'] = models_path

In [7]:
# Use --help to see what command line options are available:
!python -m official.wide_deep.census_main --help

W0618 02:20:33.613071 140282108643200 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_main.py:114: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0618 02:20:33.613363 140282108643200 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_main.py:114: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Train DNN on census income dataset.
flags:

/content/models/official/wide_deep/census_main.py:
  -bs,--batch_size:
    Batch size for training and evaluation. When using multiple gpus, this is
    the
    global batch size for all devices. For example, if the batch size is 32 and
    there are 4 GPUs, each GPU will get 8 examples on each step.
    (default: '40')
    (an integer)
  --[no]clean:
    If set, model_dir will be removed if it exists.
    (default: 'false')
  -dd,--data_dir:
    The location of the input data.
    (default: '/tmp/census_da

In [8]:
# run model
!python -m official.wide_deep.census_main --model_type=wide --train_epochs=2

W0618 02:21:41.690503 140374330308480 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_main.py:114: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0618 02:21:41.690805 140374330308480 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_main.py:114: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0618 02:21:41.693827 140374330308480 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:78: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0618 02:21:41.694007 140374330308480 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_dataset.py:81: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0618 02:21:41.694878 140374330308480 deprecation_wrapper.py:119] From /content/models/official/wide_deep/census_main.py:49: The name tf

In [9]:
# Read US census data
!ls /tmp/census_data/

adult.data  adult.test


In [0]:
train_file = '/tmp/census_data/adult.data'
test_file = '/tmp/census_data/adult.test'

In [11]:
import pandas as pd

train_df = pd.read_csv(train_file, names=census_dataset._CSV_COLUMNS)
test_df = pd.read_csv(test_file, names=census_dataset._CSV_COLUMNS)

train_df.tail()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [12]:
# 위에서 eager execution enable했기 때문에 쉽게 데이터셋 보기 가능
# tf.data.Dataset으로 slicing하는건 데이터가 작기때문에 가능
def easy_input_function(df, label_key, num_epochs, shuffle, batch_size):
  label = df[label_key]
  ds = tf.data.Dataset.from_tensor_slices((dict(df),label))

  if shuffle:
    ds = ds.shuffle(10000)

  ds = ds.batch(batch_size).repeat(num_epochs)

  return ds

ds = easy_input_function(train_df, label_key='income_bracket', num_epochs=5, shuffle=True, batch_size=10)

for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys())[:5])
  print()
  print('A batch of Ages  :', feature_batch['age'])
  print()
  print('A batch of Labels:', label_batch )

Some feature keys: ['age', 'workclass', 'fnlwgt', 'education', 'education_num']

A batch of Ages  : tf.Tensor([22 33 23 36 45 18 41 53 45 27], shape=(10,), dtype=int32)

A batch of Labels: tf.Tensor(
[b'<=50K' b'>50K' b'<=50K' b'>50K' b'<=50K' b'<=50K' b'<=50K' b'>50K'
 b'<=50K' b'<=50K'], shape=(10,), dtype=string)


In [23]:
# 데이터가 커지면....tf.decode_csv와 tf.data.TextLineDataset를 사용한다. (이 둘을 사용한게 input_fn)
# 위와 같은 결과가 나온다.
# input_fn을 사용하여 batch와 epochs를 지정하여 계속 뽑을 수 있다.
import inspect
print(inspect.getsource(census_dataset.input_fn))

ds = census_dataset.input_fn(train_file, num_epochs=5, shuffle=True, batch_size=10)

for feature_batch, label_batch in ds.take(1):
  print('Feature keys:', list(feature_batch.keys())[:5])
  print()
  print('Age batch   :', feature_batch['age'])
  print()
  print('Label batch :', label_batch )

def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have run census_dataset.py and '
      'set the --data_dir argument to the correct path.' % data_file)

  def parse_csv(value):
    tf.logging.info('Parsing {}'.format(data_file))
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('income_bracket')
    classes = tf.equal(labels, '>50K')  # binary classification
    return features, classes

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = 

In [0]:
# Because Estimators expect an input_fn that takes no arguments, 
# we typically wrap configurable input function into an obejct with the expected signature. 
# For this notebook configure the train_inpf to iterate over the data twice:
import functools

train_inpf = functools.partial(census_dataset.input_fn, train_file, num_epochs=2, shuffle=True, batch_size=64)
test_inpf = functools.partial(census_dataset.input_fn, test_file, num_epochs=1, shuffle=False, batch_size=64)

In [19]:
# model only using a column
classifier = tf.estimator.LinearClassifier(feature_columns=[fc.numeric_column('age')])
classifier.train(train_inpf)
result = classifier.evaluate(test_inpf)

clear_output()
print(result)

{'accuracy': 0.7590443, 'accuracy_baseline': 0.76377374, 'auc': 0.67830986, 'auc_precision_recall': 0.31138018, 'average_loss': 0.52304345, 'label/mean': 0.23622628, 'loss': 33.394787, 'precision': 0.14678898, 'prediction/mean': 0.2417537, 'recall': 0.0041601663, 'global_step': 1018}


In [21]:
# Similarly, we can define a NumericColumn for each continuous feature column that we want to use in the model:
# 이런식으로 numerical columns들만 뽑을 수도 있음.
age = fc.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

my_numeric_columns = [age,education_num, capital_gain, capital_loss, hours_per_week]

fc.input_layer(feature_batch, my_numeric_columns).numpy()

W0618 02:44:06.778625 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column.py:205: NumericColumn._get_dense_tensor (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0618 02:44:06.781412 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column.py:2115: NumericColumn._transform_feature (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0618 02:44:06.785332 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_c

array([[27.,  0.,  0., 11., 40.],
       [29.,  0.,  0.,  6., 40.],
       [22.,  0.,  0., 11., 75.],
       [34.,  0.,  0., 10., 40.],
       [38.,  0.,  0.,  9., 40.],
       [45.,  0.,  0.,  9., 40.],
       [24.,  0.,  0., 10., 40.],
       [38.,  0.,  0., 13., 40.],
       [40.,  0.,  0., 11., 40.],
       [34.,  0.,  0., 12., 50.]], dtype=float32)

In [22]:
classifier = tf.estimator.LinearClassifier(feature_columns=my_numeric_columns)
classifier.train(train_inpf)

result = classifier.evaluate(test_inpf)

clear_output()

for key,value in sorted(result.items()):
  print('%s: %s' % (key, value))

accuracy: 0.78250724
accuracy_baseline: 0.76377374
auc: 0.71121407
auc_precision_recall: 0.5095864
average_loss: 7.3911877
global_step: 1018
label/mean: 0.23622628
loss: 471.9056
precision: 0.61526835
prediction/mean: 0.26402935
recall: 0.21164846


In [25]:
# categorical columns들에 대해선...
# 다음과 같이 뽑을 수 있음.
relationship = fc.categorical_column_with_vocabulary_list(
    'relationship',
    ['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'])

fc.input_layer(feature_batch, [age, fc.indicator_column(relationship)])

W0618 02:49:16.828300 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column.py:205: IndicatorColumn._get_dense_tensor (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0618 02:49:16.831818 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column.py:2115: IndicatorColumn._transform_feature (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0618 02:49:16.835200 140493422073728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/featu

<tf.Tensor: id=5182, shape=(10, 7), dtype=float32, numpy=
array([[22.,  0.,  0.,  0.,  0.,  0.,  1.],
       [44.,  0.,  0.,  1.,  0.,  0.,  0.],
       [48.,  1.,  0.,  0.,  0.,  0.,  0.],
       [68.,  1.,  0.,  0.,  0.,  0.,  0.],
       [39.,  1.,  0.,  0.,  0.,  0.,  0.],
       [37.,  1.,  0.,  0.,  0.,  0.,  0.],
       [56.,  1.,  0.,  0.,  0.,  0.,  0.],
       [42.,  1.,  0.,  0.,  0.,  0.,  0.],
       [28.,  0.,  0.,  0.,  0.,  1.,  0.],
       [18.,  0.,  0.,  0.,  1.,  0.,  0.]], dtype=float32)>

In [26]:
# 위는 category갯수를 아는 경우에 해당되고,
# 모를 땐 다음을 활용한다. categorical_column_with_hash_bucket
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    'occupation', hash_bucket_size=1000)

for item in feature_batch['occupation'].numpy():
    print(item.decode())

Priv-house-serv
Exec-managerial
Adm-clerical
?
Exec-managerial
Transport-moving
Adm-clerical
Craft-repair
Craft-repair
Sales


In [0]:
education = tf.feature_column.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])


my_categorical_columns = [relationship, occupation, education, marital_status, workclass]

In [28]:
classifier = tf.estimator.LinearClassifier(feature_columns=my_numeric_columns+my_categorical_columns)
classifier.train(train_inpf)
result = classifier.evaluate(test_inpf)

clear_output()

for key,value in sorted(result.items()):
  print('%s: %s' % (key, value))

accuracy: 0.82673055
accuracy_baseline: 0.76377374
auc: 0.87491494
auc_precision_recall: 0.6558813
average_loss: 1.6820824
global_step: 1018
label/mean: 0.23622628
loss: 107.39602
precision: 0.63922846
prediction/mean: 0.25356495
recall: 0.6118045


In [0]:
# to be continue....